1. Nuget Install

In [ ]:
#r "nuget:Microsoft.ML,*-*"
#r "nuget:Microsoft.Data.Analysis, *-*"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*"
#r "nuget:Microsoft.ML.FastTree, *-*"

Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22313.1 Microsoft.DotNet.Interactive.ExtensionLab, 1.0.0-beta.22504.6 Microsoft.ML, 2.0.0-preview.22313.1 Microsoft.ML.FastTree, 2.0.0-preview.22313.1

In [ ]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Data.Analysis;

using System.IO;
using System.Text;

2. Data Models

In [ ]:
public class PenguinData
{
    [LoadColumn(0)]
    public bool Label {get;set;}

    [LoadColumn(2)]
    public float CulmenLength {get;set;}

    [LoadColumn(3)]
    public float CulmenDepth {get;set;}
}

public class PenguinPrediction
{
    public bool PredictedLabel{get;set;}
}

3. Load and Split Data

In [ ]:
var mlContext = new MLContext();
var trainingDataView = mlContext.Data.LoadFromTextFile<PenguinData>("../../ML.Net.Training/Datasets/penguins_size_binary.csv", hasHeader: true, separatorChar: ',');
var dataSplit = mlContext.Data.TrainTestSplit(trainingDataView, testFraction: 0.3);

In [ ]:
dataSplit.TestSet.ToTabularDataResource().Display();

4. Train Model

In [ ]:
var model = mlContext.BinaryClassification.Trainers.FastTree(
    numberOfLeaves: 3,
    numberOfTrees: 3,
    learningRate: 0.1
);

var pipeline = mlContext.Transforms.Concatenate("Features",
                                                nameof(PenguinData.CulmenDepth),
                                                nameof(PenguinData.CulmenLength))
                .Append(mlContext.Transforms.NormalizeMinMax("Features", "Features"))
                .Append(model);

var trainedModel = pipeline.Fit(dataSplit.TrainSet);

5. Evalute Model

In [ ]:
var testSet = trainedModel.Transform(dataSplit.TestSet);
var metrics = mlContext.BinaryClassification.Evaluate(testSet);

In [ ]:
metrics.Accuracy

0.9583333333333334

6. Save Model

In [ ]:
mlContext.Model.Save(trainedModel, dataSplit.TrainSet.Schema, "./DesicionTreesModel.mdl");

7. Load and Predict

In [ ]:
var newSample = new PenguinData
{
    CulmenDepth = 18.7f,
    CulmenLength = 39.3f
};

using (var stream = new FileStream("./DesicionTreesModel.mdl", FileMode.Open, FileAccess.Read, FileShare.Read))
{
    var loadedModel = mlContext.Model.Load(stream, out _);
    var predictionEngine = mlContext.Model.CreatePredictionEngine<PenguinData, PenguinPrediction>(loadedModel);
    var results = predictionEngine.Predict(newSample);

    results.Display();
}

PredictedLabel
True
